In [1]:
import json
import os.path
import time
import logging
import datetime

import requests
import geojson
import tweepy

import IPython.display

logger = logging.getLogger('notebook')
logger.setLevel(logging.INFO)


In [10]:
# secret keys are stored here
keys = json.load(open(os.path.expanduser("~/.twitter.json")))['notebook-example']

# you can generate keys here:
# https://apps.twitter.com/

In [3]:
# now we can load the access keys
auth = tweepy.OAuthHandler(keys['consumer_key'], keys['consumer_secret'])
auth.set_access_token(keys['access_key'], keys['access_secret'])
api = tweepy.API(auth)

In [4]:
# this should display your name
user = api.me()
user.name

'SiggyF'

In [5]:
public_tweets = api.home_timeline(count=2)
for tweet in public_tweets:
    print(tweet.created_at, tweet.text)
    

2017-06-26 12:00:20 Vanaf 1 september zijn er wijzigingen bij de wijkmanagers https://t.co/CrR3cj10rm https://t.co/hES60uzyII
2017-06-26 11:50:34 New quantile values for Rhine discharges up to 1/100000 per year at #eva2017 by Ferdinand Diermanse. https://t.co/lKMkZK8xmz


In [6]:
# display a tweet inline
IPython.display.HTML(api.get_oembed(id=tweet.id, hide_media=False)['html'])

In [7]:
from tweepy import Stream
from tweepy.streaming import StreamListener

In [8]:
# now let's start listening for events
class PrintListener(tweepy.StreamListener):
    def on_status(self, status):
        print(status.text)
        
print_listener = PrintListener()
stream = Stream(api.auth, print_listener)
stream.filter(track=['flood', 'overstroming'], async=True)

RT @tumblrcunuz: ahahah efsane flood ya :D https://t.co/SqjBe4ifN8
RT @tumblrcunuz: ahahah efsane flood ya :D https://t.co/SqjBe4ifN8
RT @tumblrcunuz: ahahah efsane flood ya :D https://t.co/SqjBe4ifN8
RT @OkKelly22: These are the men we are allowing to flood into Europe
Tribal, violent, deadly➖and everyone too scared to object

💀
https://…
RT @tumblrcunuz: ahahah efsane flood ya :D https://t.co/SqjBe4ifN8
RT @tumblrcunuz: ahahah efsane flood ya :D https://t.co/SqjBe4ifN8
Flood Map - locate your place and risk. https://t.co/ejKnCDCFK3
RT @tumblrcunuz: ahahah efsane flood ya :D https://t.co/SqjBe4ifN8
RT @CADroughtMonito: Parks are pork, but flood control saves lives. That's the tension just under the surface of the LA River https://t.co/…
RT @tumblrcunuz: ahahah efsane flood ya :D https://t.co/SqjBe4ifN8
RT @tumblrcunuz: ahahah efsane flood ya :D https://t.co/SqjBe4ifN8
RT @tumblrcunuz: ahahah efsane flood ya :D https://t.co/SqjBe4ifN8


In [9]:
stream.disconnect()

RT @tumblrcunuz: ahahah efsane flood ya :D https://t.co/SqjBe4ifN8
